In [201]:
from pyspark.sql.session import SparkSession

In [202]:
from pyspark.context import SparkContext

In [203]:
from pyspark.sql.functions import *

In [204]:
from pyspark.sql.types import *

In [205]:
from pyspark.sql import *
from pyspark.sql import functions as F

In [206]:
schema = StructType(
    [
        StructField("id", IntegerType(), True),
        StructField("order_status", StringType(), True),
        StructField("order_products_value", DoubleType(), True),
        StructField("order_freight_value", DoubleType(), True),
        StructField("order_items_qty", IntegerType(), True),
        StructField("customer_city", StringType(), True),
        StructField("customer_state", StringType(), True),
        StructField("customer_zip_code_prefix", IntegerType(), True),
        StructField("product_name_length", IntegerType(), True),
        StructField("product_description_length", IntegerType(), True),
        StructField("product_photos_qty", IntegerType(), True),
        StructField("review_score", IntegerType(), True),
        StructField("order_purchase_timestamp", StringType(), True),
        StructField("order_approved_at", StringType(), True),
        StructField("order_delivered_customer_date", StringType(), True),
    ]
)

In [207]:
spark = SparkSession.builder.appName("Project_1").getOrCreate()
#was having problem with converting string to date hence used older version of spark
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

In [208]:
#use the path to your file 
df1 =  spark.read.csv("C:/Users/16072/Downloads/olist_public_dataset.csv", header=False, schema=schema)
df1.show(5)

+---+------------+--------------------+-------------------+---------------+--------------+--------------+------------------------+-------------------+--------------------------+------------------+------------+------------------------+-----------------+-----------------------------+
| id|order_status|order_products_value|order_freight_value|order_items_qty| customer_city|customer_state|customer_zip_code_prefix|product_name_length|product_description_length|product_photos_qty|review_score|order_purchase_timestamp|order_approved_at|order_delivered_customer_date|
+---+------------+--------------------+-------------------+---------------+--------------+--------------+------------------------+-------------------+--------------------------+------------------+------------+------------------------+-----------------+-----------------------------+
|  1|   delivered|                79.0|               17.8|              1|      Luziania|            GO|                     728|                 50| 

In [209]:
df1.printSchema()


root
 |-- id: integer (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_products_value: double (nullable = true)
 |-- order_freight_value: double (nullable = true)
 |-- order_items_qty: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- product_name_length: integer (nullable = true)
 |-- product_description_length: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- review_score: integer (nullable = true)
 |-- order_purchase_timestamp: string (nullable = true)
 |-- order_approved_at: string (nullable = true)
 |-- order_delivered_customer_date: string (nullable = true)



In [210]:
df1 = df1.withColumn("order_purchase_date", to_date(df1.order_purchase_timestamp, "dd/mm/yy HH:mm"))
df1 = df1.withColumn("order_approval_date", to_date(df1.order_approved_at, "dd/mm/yy HH:mm"))
df1 = df1.withColumn("order_delivered_date", to_date(df1.order_delivered_customer_date, "dd/mm/yy HH:mm"))
df1 = df1.withColumn("purchase_dayofyear", dayofyear(df1["order_purchase_date"]))
df1 = df1.withColumn("approved_dayofyear", dayofyear(df1["order_approval_date"]))
df1 = df1.withColumn("delivered_dayofyear", dayofyear(df1["order_delivered_date"]))
df1 = df1.withColumn("approval_daily_time_taken", df1["approved_dayofyear"] - df1["purchase_dayofyear"])
df1 = df1.withColumn("delivery_daily_time_taken", df1["delivered_dayofyear"] - df1["approved_dayofyear"])


In [211]:
#ts = to_timestamp("order_purchase_timestamp","dd/MM/yy  HH:mm")
#df1 = df1.withColumn("order_purchase_timestamp",ts)
#ts1 = to_timestamp("order_approved_at","dd/MM/yy  HH:mm")
#df1 = df1.withColumn("order_approved_at",ts1)
#ts2 = to_timestamp("order_delivered_customer_date","dd/MM/yy  HH:mm")
#df1 = df1.withColumn("order_delivered_customer_date",ts2)

In [212]:
df1.printSchema()

root
 |-- id: integer (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_products_value: double (nullable = true)
 |-- order_freight_value: double (nullable = true)
 |-- order_items_qty: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- product_name_length: integer (nullable = true)
 |-- product_description_length: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- review_score: integer (nullable = true)
 |-- order_purchase_timestamp: string (nullable = true)
 |-- order_approved_at: string (nullable = true)
 |-- order_delivered_customer_date: string (nullable = true)
 |-- order_purchase_date: date (nullable = true)
 |-- order_approval_date: date (nullable = true)
 |-- order_delivered_date: date (nullable = true)
 |-- purchase_dayofyear: integer (nullable = true)
 |-- approved_dayofyear: integer (nullable = 

In [213]:
df1.show()

+---+------------+--------------------+-------------------+---------------+-------------------+--------------+------------------------+-------------------+--------------------------+------------------+------------+------------------------+-----------------+-----------------------------+-------------------+-------------------+--------------------+------------------+------------------+-------------------+-------------------------+-------------------------+
| id|order_status|order_products_value|order_freight_value|order_items_qty|      customer_city|customer_state|customer_zip_code_prefix|product_name_length|product_description_length|product_photos_qty|review_score|order_purchase_timestamp|order_approved_at|order_delivered_customer_date|order_purchase_date|order_approval_date|order_delivered_date|purchase_dayofyear|approved_dayofyear|delivered_dayofyear|approval_daily_time_taken|delivery_daily_time_taken|
+---+------------+--------------------+-------------------+---------------+-------

In [214]:
#Daily Insights: Sales: a.) Total sales
total = (df1.where(col("order_status")=="delivered").agg(sum("order_products_value")))

In [215]:
total.show()


+-------------------------+
|sum(order_products_value)|
+-------------------------+
|     1.2463991470001653E7|
+-------------------------+



In [216]:
#Daily Insights: Sales: b.) Total sales in each customer city
total_city_wise = (df1.where(col("order_status")=="delivered").groupBy("customer_city","order_purchase_date").agg(sum("order_products_value")).orderBy(desc(sum("order_products_value"))))

In [217]:
total_city_wise.show()

+-------------+-------------------+-------------------------+
|customer_city|order_purchase_date|sum(order_products_value)|
+-------------+-------------------+-------------------------+
|    Sao Paulo|         2017-01-24|        38429.24999999996|
|    Sao Paulo|         2018-01-15|        37464.58999999997|
|    Sao Paulo|         2018-01-22|        34589.61999999997|
|    Sao Paulo|         2018-01-17|        33850.34999999998|
|    Sao Paulo|         2018-01-08|        33396.04999999997|
|    Sao Paulo|         2018-01-02|        32917.91999999997|
|    Sao Paulo|         2018-01-20|        32874.87999999997|
|    Sao Paulo|         2018-01-03|        31668.32999999997|
|    Sao Paulo|         2018-01-07|        31401.03999999997|
|    Sao Paulo|         2018-01-16|       31181.549999999974|
|    Sao Paulo|         2018-01-14|       31123.859999999975|
|    Sao Paulo|         2017-01-29|        30931.78999999998|
|    Sao Paulo|         2018-01-19|        30373.95999999997|
|    Sao

In [218]:
#Daily Insights: Sales: c.) Total sales in each customer state
total_state_wise = (df1.where(col("order_status")=="delivered").groupBy("customer_state","order_purchase_date").agg(sum("order_products_value")).orderBy(desc("sum(order_products_value)")))

In [219]:
total_state_wise.show()

+--------------+-------------------+-------------------------+
|customer_state|order_purchase_date|sum(order_products_value)|
+--------------+-------------------+-------------------------+
|            SP|         2017-01-24|       106880.75000000019|
|            SP|         2018-01-16|        105040.3600000002|
|            SP|         2018-01-22|         95857.3400000002|
|            SP|         2018-01-15|        95691.33000000019|
|            SP|         2018-01-14|        95463.86000000015|
|            SP|         2018-01-05|         95331.7600000002|
|            SP|         2018-01-03|        93313.03000000025|
|            SP|         2018-01-06|        91938.58000000022|
|            SP|         2018-01-07|        91411.18000000018|
|            SP|         2018-01-01|        90869.40000000014|
|            SP|         2018-01-08|        90313.09000000019|
|            SP|         2018-01-17|        89625.98000000017|
|            SP|         2018-01-19|        88255.42000

In [220]:
# Orders: a.) Total number of orders sold
total_orders_sold=(df1.where(col("order_status")=="delivered").groupBy("order_purchase_date").agg(count("id")).orderBy(desc("count(id)")))

In [221]:
total_orders_sold.show()

+-------------------+---------+
|order_purchase_date|count(id)|
+-------------------+---------+
|         2017-01-24|     2284|
|         2018-01-16|     2098|
|         2018-01-15|     2051|
|         2018-01-07|     1901|
|         2018-01-04|     1894|
|         2018-01-03|     1893|
|         2018-01-14|     1891|
|         2018-01-18|     1888|
|         2018-01-19|     1880|
|         2018-01-08|     1868|
|         2018-01-02|     1861|
|         2018-01-09|     1847|
|         2018-01-05|     1846|
|         2018-01-06|     1845|
|         2018-01-20|     1814|
|         2018-01-22|     1804|
|         2018-01-13|     1763|
|         2018-01-11|     1757|
|         2018-01-17|     1752|
|         2018-01-21|     1748|
+-------------------+---------+
only showing top 20 rows



In [222]:
#Orders: b.) City wise order distribution
total_orders_sold_city_wise=(df1.where(col("order_status")=="delivered").groupBy("customer_city","order_purchase_date").agg(count("id")).orderBy(desc("count(id)")))

In [223]:
total_orders_sold_city_wise.show()

+-------------+-------------------+---------+
|customer_city|order_purchase_date|count(id)|
+-------------+-------------------+---------+
|    Sao Paulo|         2017-01-24|      320|
|    Sao Paulo|         2018-01-02|      284|
|    Sao Paulo|         2018-01-07|      284|
|    Sao Paulo|         2018-01-15|      283|
|    Sao Paulo|         2018-01-16|      283|
|    Sao Paulo|         2018-01-20|      277|
|    Sao Paulo|         2018-01-03|      270|
|    Sao Paulo|         2018-01-19|      265|
|    Sao Paulo|         2018-01-08|      263|
|    Sao Paulo|         2018-01-04|      262|
|    Sao Paulo|         2018-01-14|      258|
|    Sao Paulo|         2018-01-17|      257|
|    Sao Paulo|         2018-01-05|      251|
|    Sao Paulo|         2018-01-09|      250|
|    Sao Paulo|         2018-01-18|      250|
|    Sao Paulo|         2018-01-21|      250|
|    Sao Paulo|         2018-01-22|      250|
|    Sao Paulo|         2018-01-13|      246|
|    Sao Paulo|         2018-01-06

In [224]:
#Orders c.) State wise order distribution
total_orders_sold_state_wise=(df1.where(col("order_status")=="delivered").groupBy("customer_state","order_purchase_date").agg(count("id")).orderBy(desc("count(id)")))
total_orders_sold_state_wise.show()

+--------------+-------------------+---------+
|customer_state|order_purchase_date|count(id)|
+--------------+-------------------+---------+
|            SP|         2017-01-24|      921|
|            SP|         2018-01-16|      872|
|            SP|         2018-01-15|      824|
|            SP|         2018-01-19|      811|
|            SP|         2018-01-05|      806|
|            SP|         2018-01-03|      794|
|            SP|         2018-01-02|      793|
|            SP|         2018-01-14|      780|
|            SP|         2018-01-07|      778|
|            SP|         2018-01-04|      775|
|            SP|         2018-01-18|      774|
|            SP|         2018-01-08|      771|
|            SP|         2018-01-09|      765|
|            SP|         2018-01-22|      757|
|            SP|         2018-01-21|      756|
|            SP|         2018-01-06|      750|
|            SP|         2018-01-20|      743|
|            SP|         2018-01-13|      735|
|            

In [225]:
#orders d.) Average Review score per order
total_orders_average_review_score=(df1.where(col("order_status")=="delivered").groupBy("order_purchase_date").agg(avg("review_score"), count("id")).orderBy(desc("count(id)")))
total_orders_average_review_score.show()

+-------------------+------------------+---------+
|order_purchase_date| avg(review_score)|count(id)|
+-------------------+------------------+---------+
|         2017-01-24| 4.116024518388792|     2284|
|         2018-01-16|4.1549094375595805|     2098|
|         2018-01-15| 4.077523159434422|     2051|
|         2018-01-07| 4.105207785376118|     1901|
|         2018-01-04| 4.150475184794087|     1894|
|         2018-01-03| 4.161648177496038|     1893|
|         2018-01-14| 4.144368059227922|     1891|
|         2018-01-18| 4.132415254237288|     1888|
|         2018-01-19| 4.131382978723404|     1880|
|         2018-01-08| 4.111884368308351|     1868|
|         2018-01-02|  4.15260612573885|     1861|
|         2018-01-09| 4.121819166215484|     1847|
|         2018-01-05| 4.076381365113759|     1846|
|         2018-01-06|4.0910569105691055|     1845|
|         2018-01-20| 4.071664829106946|     1814|
|         2018-01-22|4.1042128603104215|     1804|
|         2018-01-13|4.05672149

In [226]:
#orders e.) Average Freight charges per order
total_orders_average_freight_value=(df1.where(col("order_status")=="delivered").groupBy("order_purchase_date").agg(avg("order_freight_value"), count("id")).orderBy(desc("count(id)")))
total_orders_average_freight_value.show()

+-------------------+------------------------+---------+
|order_purchase_date|avg(order_freight_value)|count(id)|
+-------------------+------------------------+---------+
|         2017-01-24|       21.25086252189141|     2284|
|         2018-01-16|      21.199065776930425|     2098|
|         2018-01-15|       21.89894685519258|     2051|
|         2018-01-07|       21.50054182009471|     1901|
|         2018-01-04|      20.711282998944057|     1894|
|         2018-01-03|      21.183993660855794|     1893|
|         2018-01-14|      21.282511898466424|     1891|
|         2018-01-18|       21.99204978813562|     1888|
|         2018-01-19|      22.192643617021297|     1880|
|         2018-01-08|      21.295583511777313|     1868|
|         2018-01-02|       21.94364320257928|     1861|
|         2018-01-09|      21.061656740660542|     1847|
|         2018-01-05|       21.75380823401951|     1846|
|         2018-01-06|      22.067257452574545|     1845|
|         2018-01-20|      22.3

In [227]:
#orders f.) Average time taken to approve the orders
total_orders_average_approve_time=(df1.where(col("order_status")=="delivered").groupBy("order_purchase_date").agg(avg(df1["approved_dayofyear"] - df1["purchase_dayofyear"]).alias("approval_time"), count("id")).orderBy(desc("count(id)")))
total_orders_average_approve_time.show()

+-------------------+-------------------+---------+
|order_purchase_date|      approval_time|count(id)|
+-------------------+-------------------+---------+
|         2017-01-24| 0.5330705212439772|     2284|
|         2018-01-16| 0.4343675417661098|     2098|
|         2018-01-15|      0.43798828125|     2051|
|         2018-01-07| 0.4045358649789029|     1901|
|         2018-01-04| 0.5491024287222809|     1894|
|         2018-01-03| 0.6723133933298041|     1893|
|         2018-01-14|0.45026455026455026|     1891|
|         2018-01-18| 0.4376657824933687|     1888|
|         2018-01-19| 0.5803191489361702|     1880|
|         2018-01-08|0.40492769148366364|     1868|
|         2018-01-02|0.46559139784946235|     1861|
|         2018-01-09| 0.5225420966865834|     1847|
|         2018-01-05| 0.4596205962059621|     1846|
|         2018-01-06| 0.5276872964169381|     1845|
|         2018-01-20| 0.7009397457158651|     1814|
|         2018-01-22| 0.6988352745424293|     1804|
|         20

In [228]:
#orders g.) Average time taken to deliver the orders
total_orders_average_delivery_time=(df1.where(col("order_status")=="delivered").groupBy("order_purchase_date").agg(avg(df1["delivered_dayofyear"] - df1["approved_dayofyear"]).alias("averag_delivery_time"), count("id")).orderBy(desc("count(id)")))
total_orders_average_delivery_time.show()

+-------------------+--------------------+---------+
|order_purchase_date|averag_delivery_time|count(id)|
+-------------------+--------------------+---------+
|         2017-01-24| -11.449255751014885|     2284|
|         2018-01-16|  3.0355058365758754|     2098|
|         2018-01-15|    4.26392373306573|     2051|
|         2018-01-07|   8.301724137931034|     1901|
|         2018-01-04|   9.260963724959394|     1894|
|         2018-01-03|   9.462702702702703|     1893|
|         2018-01-14|  4.6879049676025915|     1891|
|         2018-01-18|  1.0016216216216216|     1888|
|         2018-01-19| -0.4371941272430669|     1880|
|         2018-01-08|   8.484665936473165|     1868|
|         2018-01-02|   9.898789878987898|     1861|
|         2018-01-09|   7.617647058823529|     1847|
|         2018-01-05|   8.974358974358974|     1846|
|         2018-01-06|   8.851216814159292|     1845|
|         2018-01-20|  -2.089225589225589|     1814|
|         2018-01-22|  -5.248868778280543|    

In [229]:
#weekly Sales a.) total sales
total_week_sales=(df1.where(col("order_status")=="delivered").groupBy(weekofyear(df1["order_purchase_date"]).alias("week_of_year")).agg(sum("order_products_value")))
total_week_sales.show()

+------------+-------------------------+
|week_of_year|sum(order_products_value)|
+------------+-------------------------+
|          53|                  1019.29|
|           1|       2936587.8300000126|
|          52|       182084.67000000022|
|           3|       2932148.9799999953|
|           5|        647104.6099999909|
|           4|       2874179.8600000227|
|           2|        2890866.230000019|
+------------+-------------------------+



In [230]:
#weekly Sales b.) total sales city wise
total_week_sales_city_wise=(df1.where(col("order_status")=="delivered").groupBy(weekofyear(df1["order_purchase_date"]).alias("week_of_year"),"customer_city").agg(sum("order_products_value").alias("total_sale")))
total_week_sales_city_wise.show()

+------------+--------------------+------------------+
|week_of_year|       customer_city|        total_sale|
+------------+--------------------+------------------+
|           2|    Mata de Sao Joao|352.96999999999997|
|           2|            Sao Luis|12833.649999999994|
|           2|            Paulinia|3634.8799999999997|
|           1|          Acailandia|            297.88|
|           1|          Mogi Mirim|           1385.67|
|           4|    Feira de Santana|           5340.73|
|           3|            Altamira|1154.8899999999999|
|           4|              Andira| 543.1800000000001|
|           5|Visconde do Rio B...|             162.9|
|           4|Itapecerica da Serra|           3260.37|
|           1|             Caninde|             553.4|
|           2|            Botucatu|3017.1400000000003|
|           4|              Calmon|              49.9|
|           4|             Uberaba|3703.8600000000006|
|           3|           Caravelas|            1340.0|
|         

In [231]:
#weekly Sales c.) total sales state wise
total_week_sales_state_wise=(df1.where(col("order_status")=="delivered").groupBy(weekofyear(df1["order_purchase_date"]).alias("week_of_year"),"customer_state").agg(sum("order_products_value").alias("total_sales_state_wise")))
total_week_sales_state_wise.show()

+------------+--------------+----------------------+
|week_of_year|customer_state|total_sales_state_wise|
+------------+--------------+----------------------+
|           4|            RJ|     383042.3999999979|
|           3|            CE|     49006.90999999999|
|          52|            BA|               7354.55|
|          52|            MS|                679.46|
|           4|            MT|    32623.999999999985|
|           4|            PE|     51719.12000000003|
|           4|            PR|    140061.87000000037|
|           3|            MT|    32296.889999999978|
|          52|            MG|    20795.189999999995|
|           5|            SP|    247231.38999999978|
|           2|            GO|     59273.78000000002|
|           2|            RR|               1365.01|
|           1|            SE|    11390.419999999998|
|           5|            ES|     9639.269999999999|
|           4|            SC|     105528.5400000002|
|           1|            TO|              105

In [232]:
#weekly Orders a.) total number of orders sold
total_week_orders=(df1.where(col("order_status")=="delivered").groupBy(weekofyear(df1["order_purchase_date"]).alias("week_of_year")).agg(count("id").alias("total_orders")))
total_week_orders.show()

+------------+------------+
|week_of_year|total_orders|
+------------+------------+
|          53|           9|
|           1|       23024|
|          52|        1373|
|           3|       22952|
|           5|        5246|
|           4|       22530|
|           2|       22494|
+------------+------------+



In [233]:
#weekly Orders b.) city wise order distribution
total_week_orders_city_wise=(df1.where(col("order_status")=="delivered").groupBy(weekofyear(df1["order_purchase_date"]).alias("week_of_year"),"customer_city").agg(count("id").alias("total_orders")))
total_week_orders_city_wise.show()

+------------+--------------------+------------+
|week_of_year|       customer_city|total_orders|
+------------+--------------------+------------+
|           2|    Mata de Sao Joao|           4|
|           2|            Sao Luis|          77|
|           2|            Paulinia|          28|
|           1|          Acailandia|           2|
|           1|          Mogi Mirim|          14|
|           4|    Feira de Santana|          42|
|           3|            Altamira|           5|
|           4|              Andira|           5|
|           5|Visconde do Rio B...|           2|
|           4|Itapecerica da Serra|          25|
|           1|             Caninde|           4|
|           2|            Botucatu|          32|
|           4|              Calmon|           1|
|           4|             Uberaba|          31|
|           3|           Caravelas|           1|
|           5|              Ilheus|           3|
|           1|Livramento de Nos...|           1|
|           3|      

In [234]:
#weekly Orders c.) state wise order distribution
total_week_orders_state_wise=(df1.where(col("order_status")=="delivered").groupBy(weekofyear(df1["order_purchase_date"]).alias("week_of_year"),"customer_state").agg(count("id").alias("total_orders")))
total_week_orders_state_wise.show()

+------------+--------------+------------+
|week_of_year|customer_state|total_orders|
+------------+--------------+------------+
|           4|            RJ|        3065|
|           3|            CE|         301|
|          52|            BA|          53|
|          52|            MS|           7|
|           4|            MT|         210|
|           4|            PE|         363|
|           4|            PR|        1118|
|           3|            MT|         213|
|          52|            MG|         161|
|           5|            SP|        2128|
|           2|            GO|         425|
|           2|            RR|          11|
|           1|            SE|          70|
|           5|            ES|         102|
|           4|            SC|         852|
|           1|            TO|          70|
|           5|            PA|          63|
|           1|            SC|         797|
|           1|            PI|          80|
|           3|            AL|         115|
+----------

In [235]:
#weekly Orders d.) average review score
total_week_average_review_score=(df1.where(col("order_status")=="delivered").groupBy(weekofyear(df1["order_purchase_date"]).alias("week_of_year")).agg(avg("review_score").alias("average_score")))
total_week_average_review_score.show()

+------------+------------------+
|week_of_year|     average_score|
+------------+------------------+
|          53|3.7777777777777777|
|           1| 4.118702223766505|
|          52| 4.113619810633649|
|           3| 4.099817009410945|
|           5| 4.119329012581014|
|           4| 4.101686640035508|
|           2| 4.097581577309505|
+------------+------------------+



In [236]:
#weekly Orders e.) average freight charges per order
total_week_average_freight_value=(df1.where(col("order_status")=="delivered").groupBy(weekofyear(df1["order_purchase_date"]).alias("week_of_year")).agg(avg("order_freight_value").alias("average_freight_value")))
total_week_average_freight_value.show()

+------------+---------------------+
|week_of_year|average_freight_value|
+------------+---------------------+
|          53|   15.643333333333333|
|           1|   21.649496177901547|
|          52|    22.48302257829571|
|           3|   21.764004444057335|
|           5|   21.424613038505484|
|           4|   21.805934309809327|
|           2|    21.56005690406349|
+------------+---------------------+



In [237]:
#weekly Orders f.) average time taken to approve the orders
total_week_average_approval_time=(df1.where(col("order_status")=="delivered").groupBy(weekofyear(df1["order_purchase_date"]).alias("week_of_year")).agg(avg(weekofyear(df1["order_approval_date"]) - weekofyear(df1["order_purchase_date"])).alias("average_approval_time")))
total_week_average_approval_time.show()

+------------+---------------------+
|week_of_year|average_approval_time|
+------------+---------------------+
|          53|                -52.0|
|           1|  0.06743528388079183|
|          52|  -15.799416484318016|
|           3|  0.08234369542438003|
|           5|   2.1195797516714423|
|           4|                 0.21|
|           2|  0.06917427108835968|
+------------+---------------------+



In [238]:
#weekly Orders g.) average order delivery time
total_week_average_delivery_time=(df1.where(col("order_status")=="delivered").groupBy(weekofyear(df1["order_purchase_date"]).alias("week_of_year")).agg(avg(weekofyear(df1["order_delivered_date"]) - weekofyear(df1["order_approval_date"])).alias("average_delivery_time")))
total_week_average_delivery_time.show()

+------------+---------------------+
|week_of_year|average_delivery_time|
+------------+---------------------+
|          53|                20.75|
|           1|    1.515392209657153|
|          52|  -34.020439061317184|
|           3|   0.8787973832780716|
|           5|    -4.80685174246899|
|           4|  -0.6379578043656955|
|           2|   1.3513277989267531|
+------------+---------------------+



In [239]:
#Total Freight charges
total_week_freight_charges=(df1.where(col("order_status")=="delivered").agg(sum("order_freight_value").alias("Total_freight_value")))
total_week_freight_charges.show()

+-------------------+
|Total_freight_value|
+-------------------+
| 2117648.5499999207|
+-------------------+



In [240]:
#Freight charges distribution in each customer city
total_week_freight_charges_city_wise=(df1.where(col("order_status")=="delivered").groupBy("customer_city").agg(sum("order_freight_value").alias("Total_Freight_value")))
total_week_freight_charges_city_wise.show()

+----------------+-------------------+
|   customer_city|Total_Freight_value|
+----------------+-------------------+
|        Araruama|            2304.13|
|        Guidoval|              35.21|
|     Piranguinho|              80.92|
|     Tres Pontas| 362.59000000000003|
|Senador Guiomard|             137.11|
|       Carrancas| 48.260000000000005|
|       Fronteira|             103.19|
|          Utinga|              50.67|
|    Assis Brasil|              24.84|
|          Pianco|             195.64|
|        Macaubas|             294.86|
|      Livramento|              24.84|
|    Cristalandia|              97.62|
|        Rio Novo|              22.98|
|        Machados|             127.52|
|    Rio do Campo| 100.53999999999999|
|      Purilandia|              27.49|
|        Alambari|              75.12|
|   Guajará-Mirim|             154.69|
|           Tapes| 151.85999999999996|
+----------------+-------------------+
only showing top 20 rows

